In [1]:
!pip install yahoo_fin
!pip install requests_html
!pip install plotly.express
!pip install nbformat 

**We found the below method of downloading Yahoo Finance information from the statistics page on the internet and it seems to work. Alternativly, one could go into each of the ticker codes of Yahoo Finance page**


In [2]:
import yahoo_fin.stock_info as si #Get data
import pandas as pd #Data manipulation
from tqdm import tqdm #Make a progress bar because that's cool...
from bs4 import BeautifulSoup # to parse external data
import requests # to get data
from requests_html import HTMLSession
#import matplotlib.pyplot as plt
import plotly.express as px
import pickle

In [3]:
def getEURSTX50tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/EURO_STOXX_50')
    soup = BeautifulSoup(resp.text, 'lxml')
    tableEURSTX = soup.find(text="Ticker").find_parent("table").find('tbody').findAll('tr')[1:]

    EURSTX_tickers = []
    for row in tableEURSTX:
        ticker1 = row.findAll('td')[0].text.strip()
        EURSTX_tickers.append(ticker1)
    with open("EURSTX50tickers.pickle", "wb") as f:
        pickle.dump(EURSTX_tickers, f)
    return EURSTX_tickers

ER_tickers = getEURSTX50tickers()
ER_tickers

['ADS.DE',
 'ADYEN.AS',
 'AD.AS',
 'AI.PA',
 'AIR.PA',
 'ALV.DE',
 'ABI.BR',
 'ASML.AS',
 'CS.PA',
 'BAS.DE',
 'BAYN.DE',
 'BBVA.MC',
 'SAN.MC',
 'BMW.DE',
 'BNP.PA',
 'CRG.IR',
 'DAI.DE',
 'BN.PA',
 'DB1.DE',
 'DPW.DE',
 'DTE.DE',
 'ENEL.MI',
 'ENI.MI',
 'EL.PA',
 'FLTR.IR',
 'IBE.MC',
 'ITX.MC',
 'IFX.DE',
 'INGA.AS',
 'ISP.MI',
 'KER.PA',
 'KNEBV.HE',
 'OR.PA',
 'LIN.DE',
 'MC.PA',
 'MUV2.DE',
 'RI.PA',
 'PHIA.AS',
 'PRX.AS',
 'SAF.PA',
 'SAN.PA',
 'SAP.DE',
 'SU.PA',
 'SIE.DE',
 'STLA.MI',
 'TTE.PA',
 'UMG.AS',
 'DG.PA',
 'VOW.DE',
 'VNA.DE']

In [4]:
def valuation_measures(reload_EURSTX50=False):
       
    if reload_EURSTX50:
        EURSTX_tickers = getEURSTX50tickers()
    else:
        with open("EURSTX50tickers.pickle","rb") as f:
            EURSTX_tickers = pickle.load(f)
    ticker_stats = {}     
    for ticker in EURSTX_tickers:
        try:
            df = si.get_stats_valuation(ticker)
            df = df.iloc[:,:2]
            df.columns = ["Attribute", "Recent"]
            ticker_stats[ticker] = df
        except:
            pass
    dat = pd.concat(ticker_stats)
    dat = dat.reset_index()
    dat = dat.dropna()
    del dat["level_1"]
    dat.columns = ["Ticker", "Attribute", "Recent"]
    dat.to_csv('df1.csv')
    return dat

In [5]:
df1 = valuation_measures()
df1

,Ticker,Attribute,Recent
0,ADS.DE,Market Cap (intraday),46.54B
1,ADS.DE,Enterprise Value,47.38B
2,ADS.DE,Trailing P/E,32.69
3,ADS.DE,Forward P/E,24.94
4,ADS.DE,PEG Ratio (5 yr expected),0.59
...,...,...,...
310,MC.PA,PEG Ratio (5 yr expected),1.17
311,MC.PA,Price/Sales (ttm),6.53
312,MC.PA,Price/Book (mrq),8.70
313,MC.PA,Enterprise Value/Revenue,7.06


In [6]:
def extra_stats(reload_EURSTX50=False):
    
    if reload_EURSTX50:
        EURSTX_tickers = getEURSTX50tickers()
    else:
        with open("EURSTX50tickers.pickle","rb") as f:
            EURSTX_tickers = pickle.load(f)
    ticker_extra_stats = {}
    for ticker in tqdm(ER_tickers):
        try:
            ticker_extra_stats[ticker] = si.get_stats(ticker)
        except:
            pass
    dat2 = pd.concat(ticker_extra_stats)
    dat2 = dat2.reset_index()
    dat2 = dat2.dropna()
    del dat2["level_1"]
    dat2.columns = ["Ticker", "Attribute", "Value"]
    dat2.to_csv('df2.csv')
    return dat2

In [7]:
df2 = extra_stats()
df2

100%|██████████| 50/50 [02:32<00:00,  3.04s/it]


,Ticker,Attribute,Value
0,ADS.DE,Beta (5Y Monthly),0.89
3,ADS.DE,52 Week High 3,336.25
4,ADS.DE,52 Week Low 3,231.55
5,ADS.DE,50-Day Moving Average 3,251.12
6,ADS.DE,200-Day Moving Average 3,282.58
...,...,...,...
2349,VNA.DE,Total Debt/Equity (mrq),162.57
2350,VNA.DE,Current Ratio (mrq),0.39
2351,VNA.DE,Book Value Per Share (mrq),47.85
2352,VNA.DE,Operating Cash Flow (ttm),1.44B


In [8]:
print(df1)

     Ticker                  Attribute  Recent
0    ADS.DE      Market Cap (intraday)  46.54B
1    ADS.DE           Enterprise Value  47.38B
2    ADS.DE               Trailing P/E   32.69
3    ADS.DE                Forward P/E   24.94
4    ADS.DE  PEG Ratio (5 yr expected)    0.59
..      ...                        ...     ...
310   MC.PA  PEG Ratio (5 yr expected)    1.17
311   MC.PA          Price/Sales (ttm)    6.53
312   MC.PA           Price/Book (mrq)    8.70
313   MC.PA   Enterprise Value/Revenue    7.06
314   MC.PA    Enterprise Value/EBITDA   19.34

[246 rows x 3 columns]


In [9]:
print(df2)

      Ticker                     Attribute   Value
0     ADS.DE             Beta (5Y Monthly)    0.89
3     ADS.DE                52 Week High 3  336.25
4     ADS.DE                 52 Week Low 3  231.55
5     ADS.DE       50-Day Moving Average 3  251.12
6     ADS.DE      200-Day Moving Average 3  282.58
...      ...                           ...     ...
2349  VNA.DE       Total Debt/Equity (mrq)  162.57
2350  VNA.DE           Current Ratio (mrq)    0.39
2351  VNA.DE    Book Value Per Share (mrq)   47.85
2352  VNA.DE     Operating Cash Flow (ttm)   1.44B
2353  VNA.DE  Levered Free Cash Flow (ttm)   8.66B

[1821 rows x 3 columns]


In [10]:
# Convert from long dataframe to wide dataframe using the pivot() method.
df1_wide = df1.pivot(index = "Ticker", columns="Attribute", values="Recent")
df2_wide = df2.pivot(index = "Ticker", columns="Attribute", values="Value")

In [11]:
print(df1_wide)

Attribute Enterprise Value Enterprise Value/EBITDA Enterprise Value/Revenue  \
Ticker                                                                        
ABI.BR                 NaN                     NaN                      NaN   
ADS.DE              47.38B                   14.17                     2.09   
ADYEN.AS            46.92B                   94.14                    10.12   
AI.PA               84.75B                   14.32                     4.02   
AIR.PA              84.00B                   10.68                     1.53   
ALV.DE             106.40B                     NaN                     0.94   
ASML.AS            229.72B                   32.78                    12.34   
BAS.DE              78.66B                    6.87                     1.05   
BAYN.DE             81.70B                   11.26                     1.90   
BBVA.MC                NaN                     NaN                      NaN   
BMW.DE             129.87B                    5.86  

In [12]:
print(df2_wide)

Attribute % Held by Insiders 1 % Held by Institutions 1  \
Ticker                                                    
ABI.BR                  48.74%                   14.18%   
AD.AS                    0.05%                   52.48%   
ADS.DE                  10.29%                   53.12%   
ADYEN.AS                23.66%                   62.10%   
AI.PA                    2.61%                   29.97%   
AIR.PA                  25.84%                   31.54%   
ALV.DE                   0.00%                   40.18%   
ASML.AS                  0.01%                   57.12%   
BAS.DE                   0.00%                   29.05%   
BAYN.DE                  0.00%                   38.22%   
BBVA.MC                  0.02%                   35.21%   
BMW.DE                  46.74%                   21.66%   
BN.PA                    2.65%                   55.26%   
BNP.PA                  13.20%                   45.94%   
CRG.IR                   0.02%                   53.57% 

**Now we simply need to format the data into a format which is usable in relation to the graph we wish to display:**

At the moment, we have the data in very few columns and many rows. Essentially, every ticker is repeated for every attribute. We now wish to change this such that the ticker is displayed once in a row and all the attributed on a column name. 

The next problem we encounter is that there are a bunch of footnotes in the attribute descriptions. The reason for this is that on the Yahoo Finance website, the attributes may / may not have some additional information attached to them. 

However, since we are aiming to create a scheduled graph, we are not really interested in these additional footnotes. 

Next step would be to eliminate these footnote numbers. 
In order to do that we need to rename all the colums affeted and subsequently change them from strings into floats




In [75]:
# Clean up column names to remove footnote indicators

df2_wide.rename(columns = {"% Held by Insiders 1":"% Held by Insiders"}, inplace = True)
df2_wide.rename(columns = {"% Held by Institutions 1":"% Held by Institutions"}, inplace = True)
df2_wide.rename(columns = {"200-Day Moving Average 3":"200-Day Moving Average"}, inplace = True)
df2_wide.rename(columns = {"5 Year Average Dividend Yield 4":"5 Year Average Dividend Yield %"}, inplace = True)
df2_wide.rename(columns = {"50-Day Moving Average 3":"50-Day Moving Average"}, inplace = True)
df2_wide.rename(columns = {"52 Week High 3":"52 Week High"}, inplace = True)
df2_wide.rename(columns = {"52 Week Low 3":"52 Week Low"}, inplace = True)
df2_wide.rename(columns = {"52-Week Change 3":"52-Week Change %"}, inplace = True)
df2_wide.rename(columns = {"Avg Vol (10 day) 3":"Avg Vol (10 day)"}, inplace = True)
df2_wide.rename(columns = {"Avg Vol (3 month) 3":"Avg Vol (3 month)"}, inplace = True)
df2_wide.rename(columns = {"Dividend Date 3":"Dividend Date"}, inplace = True)
df2_wide.rename(columns = {"Ex-Dividend Date 4":"Ex-Dividend Date"}, inplace = True)
df2_wide.rename(columns = {"Forward Annual Dividend Rate 4":"Forward Annual Dividend Rate"}, inplace = True)
df2_wide.rename(columns = {"Forward Annual Dividend Yield 4":"Forward Annual Dividend Yield %"}, inplace = True)
df2_wide.rename(columns = {"Last Split Date 3":"Last Split Date"}, inplace = True)
df2_wide.rename(columns = {"Last Split Factor 2":"Last Split Factor"}, inplace = True)
df2_wide.rename(columns = {"Operating Margin (ttm)":"Operating Margin (ttm) %"}, inplace = True)
df2_wide.rename(columns = {"Payout Ratio 4":"Payout Ratio %"}, inplace = True)
df2_wide.rename(columns = {"Profit Margin":"Profit Margin %"}, inplace = True)
df2_wide.rename(columns = {"Quarterly Earnings Growth (yoy)":"Quarterly Earnings Growth (yoy) %"}, inplace = True)
df2_wide.rename(columns = {"Quarterly Revenue Growth (yoy)":"Quarterly Revenue Growth (yoy) %"}, inplace = True)
df2_wide.rename(columns = {"Return on Assets (ttm)":"Return on Assets (ttm) %"}, inplace = True)
df2_wide.rename(columns = {"Return on Equity (ttm)":"Return on Equity (ttm) %"}, inplace = True)
df2_wide.rename(columns = {"S&P500 52-Week Change 3":"S&P500 52-Week Change %"}, inplace = True)
df2_wide.rename(columns = {"Shares Outstanding 5":"Shares Outstanding"}, inplace = True)
df2_wide.rename(columns = {"Trailing Annual Dividend Rate 3":"Trailing Annual Dividend Rate"}, inplace = True)
df2_wide.rename(columns = {"Trailing Annual Dividend Yield 3":"Trailing Annual Dividend Yield %"}, inplace = True)

# Convert string values to float as necessary
df1_wide['Trailing P/E'] = df1_wide['Trailing P/E'].astype(float)
df1_wide['Enterprise Value/EBITDA'] = df1_wide['Enterprise Value/EBITDA'].astype(float)
df1_wide['Enterprise Value/Revenue'] = df1_wide['Enterprise Value/Revenue'].astype(float)
df1_wide['Forward P/E'] = df1_wide['Forward P/E'].astype(float)
df1_wide['PEG Ratio (5 yr expected)'] = df1_wide['PEG Ratio (5 yr expected)'].astype(float)
df1_wide['Price/Book (mrq)'] = df1_wide['Price/Book (mrq)'].astype(float)
df1_wide['Price/Sales (ttm)'] = df1_wide['Price/Sales (ttm)'].astype(float)

#df2_wide['% Held by Insiders'] = df2_wide['% Held by Insiders'].str.strip('%').astype('float') / 100.0
#df2_wide['% Held by Institutions'] = df2_wide['% Held by Institutions'].str.strip('%').astype('float') / 100.0
#df2_wide['5 Year Average Dividend Yield %'] = df2_wide['5 Year Average Dividend Yield %'].astype(float) / 100.0
df2_wide['50-Day Moving Average'] = df2_wide['50-Day Moving Average'].astype(float)
df2_wide['52 Week High'] = df2_wide['52 Week High'].astype(float)
df2_wide['52 Week Low'] = df2_wide['52 Week Low'].astype(float)
#df2_wide['52-Week Change %'] = df2_wide['52-Week Change %'].str.strip('%').astype('float') / 100.0
df2_wide['Beta (5Y Monthly)'] = df2_wide['Beta (5Y Monthly)'].astype(float)
df2_wide['Book Value Per Share (mrq)'] = df2_wide['Book Value Per Share (mrq)'].astype(float)
df2_wide['Current Ratio (mrq)'] = df2_wide['Current Ratio (mrq)'].astype(float)
df2_wide['Diluted EPS (ttm)'] = df2_wide['Diluted EPS (ttm)'].astype(float)
df2_wide['Forward Annual Dividend Rate'] = df2_wide['Forward Annual Dividend Rate'].astype(float)
#df2_wide['Forward Annual Dividend Yield %'] = df2_wide['Forward Annual Dividend Yield %'].str.strip('%').astype('float') / 100.0
#df2_wide['Operating Margin (ttm) %'] = df2_wide['Operating Margin (ttm) %'].str.strip('%').astype('float') / 100.0
#df2_wide['Payout Ratio %'] = df2_wide['Payout Ratio %'].str.strip('%').astype('float') / 100.0
#df2_wide['Profit Margin %'] = df2_wide['Profit Margin %'].str.strip('%').astype('float') / 100.0
#df2_wide['Quarterly Earnings Growth (yoy) %'] = df2_wide['Quarterly Earnings Growth (yoy) %'].str.strip('%').astype('float') / 100.0
df2_wide['Quarterly Revenue Growth (yoy) %'] = df2_wide['Quarterly Revenue Growth (yoy) %'].astype(str).str.strip('%').astype('float') / 100.0
#df2_wide['Return on Assets (ttm) %'] = df2_wide['Return on Assets (ttm) %'].str.strip('%').astype('float') / 100.0
#df2_wide['Return on Equity (ttm) %'] = df2_wide['Return on Equity (ttm) %'].str.strip('%').astype('float') / 100.0
df2_wide['Revenue Per Share (ttm)'] = df2_wide['Revenue Per Share (ttm)'].astype(float)
#df2_wide['S&P500 52-Week Change %'] = df2_wide['S&P500 52-Week Change %'].str.strip('%').astype('float') / 100.0
df2_wide['Total Cash Per Share (mrq)'] = df2_wide['Total Cash Per Share (mrq)'].astype(float)
df2_wide['Total Debt/Equity (mrq)'] = df2_wide['Total Debt/Equity (mrq)'].astype(float)
df2_wide['Trailing Annual Dividend Rate'] = df2_wide['Trailing Annual Dividend Rate'].astype(float)
#df2_wide['Trailing Annual Dividend Yield %'] = df2_wide['Trailing Annual Dividend Yield %'].str.strip('%').astype('float') / 100.0
df2_wide['200-Day Moving Average'] = df2_wide['200-Day Moving Average'].astype(float)

# Merge the dataframes into company_data
company_data = pd.concat([df1_wide, df2_wide], axis=1, join='inner')
company_data = company_data[company_data['Trailing P/E'].notna()]
company_data


Attribute,Enterprise Value,Enterprise Value/EBITDA,Enterprise Value/Revenue,Forward P/E,Market Cap (intraday),PEG Ratio (5 yr expected),Price/Book (mrq),Price/Sales (ttm),Trailing P/E,% Held by Insiders,...,Return on Equity (ttm) %,Revenue (ttm),Revenue Per Share (ttm),Shares Outstanding,Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Trailing Annual Dividend Rate,Trailing Annual Dividend Yield %
Ticker,,,,,,,,,,,,,,,,,,,,,
ABI.BR,NaN,NaN,NaN,NaN,NaN,6.35,NaN,NaN,41.64,0.4874,...,NaN,52.87B,26.37,1.74B,7.08B,3.57,90.64B,113.92,0.61,1.09%
ADS.DE,47.38B,14.17,2.09,24.94,46.54B,0.59,6.04,2.09,32.69,0.1029,...,20.05%,22.65B,116.19,195.07M,4.43B,22.95,5.27B,66.01,3.00,1.22%
ADYEN.AS,46.92B,94.14,10.12,79.37,50.12B,NaN,34.89,10.70,134.80,0.2366,...,30.49%,4.64B,152.76,30.94M,3.34B,109.83,128.36M,8.94,NaN,NaN
AI.PA,84.75B,14.32,4.02,23.98,71.58B,2.38,3.75,3.40,27.61,0.0261,...,14.07%,21.06B,44.62,473.65M,1.39B,2.94,14.61B,74.66,2.75,1.79%
AIR.PA,84.00B,10.68,1.53,21.46,87.73B,1.11,9.98,1.60,20.95,0.2584,...,76.81%,54.91B,69.96,785.85M,16.65B,21.19,14.92B,169.50,1.65,1.46%
ALV.DE,106.40B,NaN,0.94,10.21,92.09B,1.09,1.13,0.80,10.84,0.0000,...,11.23%,119.85B,288.44,408.46M,21.94B,52.89,36.25B,42.47,9.60,4.23%
ASML.AS,229.72B,32.78,12.34,33.67,233.24B,2.49,23.00,12.65,40.01,0.0001,...,43.26%,17.88B,43.31,406.47M,4.46B,10.92,4.11B,35.69,3.35,0.56%
BAS.DE,78.66B,6.87,1.05,11.32,61.98B,0.79,1.57,0.83,11.54,0.0000,...,15.26%,74.73B,81.36,918.48M,3.1B,3.37,19.78B,48.70,3.30,4.86%
BAYN.DE,81.70B,11.26,1.90,7.86,52.03B,3.01,1.67,1.21,230.26,0.0000,...,0.76%,42.96B,43.73,982.42M,8.22B,8.37,41.77B,133.55,2.00,3.72%


In [76]:
# Plot out the data
chart1 = px.scatter(company_data,
                   x='Quarterly Revenue Growth (yoy) %',
                   y='Price/Sales (ttm)',
                   size = "Trailing P/E",
                   hover_name = company_data.index,
                   title="Quarterly Revenue Growth (yoy) % vs. Price/Sales (ttm) vs. Trailing P/E",
                   width=1250,
                   height=700)

# Update chart layouts
chart1.update_layout(
    margin=dict(l=20, r=20, t=40, b=20),
    paper_bgcolor="#ffffff",
)

chart1.show()